In [45]:
import pandas as pd
import numpy as np
import torch
from collections import defaultdict
from lstm import MathLSTM
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split



In [7]:
data = pd.read_csv('arithmetic_data.csv')

In [21]:
equations = list(data.itertuples(index=False, name=None))
equations[0]

(79, '*', 28, '=', 2212)

### tokenize the inputs

In [24]:
vocab = set()
for x, op, y, eq, z in equations:
    vocab.update([str(x), op, str(y), eq])

tokenized_math = {tok: i for i, tok in enumerate(sorted(vocab))}

vocab_size = len(tokenized_math)
print(vocab_size)


105


In [38]:
def encode(eqn):
    x, op, y, eq, z = eqn
    return [tokenized_math[str(x)], tokenized_math[op], tokenized_math[str(y)], tokenized_math[eq]], float(z)

encoded = [encode(eqn) for eqn in equations]

# Build encoded tensors
input_tensor = torch.tensor([e[0] for e in encoded], dtype=torch.long)
output_tensor = torch.tensor([e[1] for e in encoded], dtype=torch.float32)

In [37]:
print(input_tensor.shape)
print(output_tensor.shape)

torch.Size([100000, 4])
torch.Size([100000])


In [46]:
model = MathLSTM(vocab_size=len(tokenized_math), embedding_dim=16, hidden_dim=32)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    input_tensor, output_tensor, test_size=0.2, random_state=42
)

In [ ]:
num_epochs = 500

for epoch in range(num_epochs):
    model.train()
    
    # Forward pass
    preds = model(X_train)
    loss = loss_fn(preds, y_train)
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Evaluate on test set every 50 epochs
    if epoch % 50 == 0 or epoch == num_epochs - 1:
        model.eval()
        with torch.no_grad():
            test_preds = model(X_test)
            test_loss = loss_fn(test_preds, y_test)
        print(f"Epoch {epoch}: Train Loss = {loss.item():.4f}, Test Loss = {test_loss.item():.4f}")
